<a href="https://colab.research.google.com/github/solcanalla/fiumark/blob/main/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo: KNN #


##Inicialización##

In [35]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

##Pre procesamiento##

In [27]:
!git clone https://github.com/solcanalla/fiumark.git
%cd fiumark
!git pull origin main
import preprocessing as pp

Cloning into 'fiumark'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 77 (delta 37), reused 36 (delta 14), pack-reused 0
Unpacking objects: 100% (77/77), done.
/content/fiumark/fiumark
From https://github.com/solcanalla/fiumark
 * branch            main       -> FETCH_HEAD
Already up to date.


Para KNN es importante escalar los datos. 

In [36]:
pd.options.display.max_columns = None

df = pp.get_dataset()
df = pp.knn_preprocessing(df)
X_train, X_test, y_train, y_test = pp.get_train_test_data(df)


##Entrenamiento##

In [37]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'n_neighbors': np.arange(1, 100),
    'weights': ['uniform', 'distance'],
    'metric': ['cosine','minkowski'],
    'algorithm':['ball_tree','kd_tree','brute']
}

clf = KNeighborsClassifier()

rgscv = RandomizedSearchCV(
    clf, params, n_iter=200, scoring='accuracy', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

In [5]:
print(f"Best score: {rgscv.best_score_}")
print(f"Best params {rgscv.best_params_}")

Best score: 0.8117647058823529
Best params {'weights': 'uniform', 'n_neighbors': 20, 'metric': 'cosine', 'algorithm': 'brute'}


##Evaluación del modelo##

In [6]:
from sklearn.metrics import classification_report

print(classification_report(y_test, rgscv.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.77      0.97      0.86        70
         1.0       0.94      0.61      0.74        51

    accuracy                           0.82       121
   macro avg       0.86      0.79      0.80       121
weighted avg       0.84      0.82      0.81       121



##Predicción##

In [58]:
X_holdout = pd.read_csv('https://drive.google.com/uc?export=download&id=1I980-_K9iOucJO26SG5_M8RELOQ5VB6A')
X = pp.knn_preprocessing(X_holdout)
prediction = pd.DataFrame(rgscv.predict(X),columns=['volveria'],dtype=int)
pred_formatted = pd.concat([X_holdout.id_usuario,prediction],axis=1)
pred_formatted.to_csv('knn.csv')
pred_formatted